# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [30]:
%matplotlib inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

In [31]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print (classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [32]:
import joblib

def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = joblib.load(filename)
    target = [i] * len(data)
    
    return data, target

In [33]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in range(len(classes)):
    data, target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)

print ('total number of data', len(all_data))

total number of data 222


In [34]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [35]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [36]:
# YOUR CODE HERE
all_data = np.array(all_data)
all_targets = np.array(all_target)
samples = all_data[permutation[:n_training_data]]
targets = all_targets[permutation[:n_training_data]]
clf.fit(samples, targets)

SVC(C=100.0, gamma=0.001)

### predicting

In [37]:
clf.predict(all_data[-1:]), all_target[-1:]

(array([10]), [10])

In [38]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [39]:
expected = []
predicted = []
# YOUR CODE HERE
expected = targets
predicted = clf.predict(samples)

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        17
           2       1.00      1.00      1.00        20
           3       1.00      1.00      1.00         7
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         4
           6       1.00      1.00      1.00        13
           7       1.00      1.00      1.00         9
           8       1.00      1.00      1.00        20
           9       1.00      1.00      1.00        10
          10       1.00      1.00      1.00        33

    accuracy                           1.00       155
   macro avg       1.00      1.00      1.00       155
weighted avg       1.00      1.00      1.00       155


Confusion matrix:
[[16  0  0  0  0  0  0  0  0  0  0]
 [ 0 17  0  0  0  0  0  0  0  0  0]
 [ 0  0 20  0  0  0  0  0  0  0  0]
 [ 0  0  0  7  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [40]:
expected = []
predicted = []
# YOUR CODE HERE
test = permutation[n_training_data:]
expected = np.asarray(all_target)[test]
predicted = clf.predict(np.asarray(all_data)[test])
evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00         2
           2       0.91      1.00      0.95        10
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         4
           5       1.00      1.00      1.00         6
           6       1.00      1.00      1.00         7
           7       1.00      0.50      0.67         2
           8       1.00      1.00      1.00         6
           9       1.00      1.00      1.00         1
          10       1.00      1.00      1.00        19

    accuracy                           0.99        67
   macro avg       0.99      0.95      0.97        67
weighted avg       0.99      0.99      0.98        67


Confusion matrix:
[[ 7  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  0  0  0  0  0  0  0  0  0]
 [ 0  0 10  0  0  0  0  0  0  0  0]
 [ 0  0  0  3  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [41]:
import joblib
ROBOT_POSE_CLF = 'robot_pose.joblib'
joblib.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [42]:
clf2 = joblib.load(ROBOT_POSE_CLF)
clf2.predict(all_data[-1:]), all_target[-1:]

(array([10]), [10])